## LP Formulation

The decision variables $P_{i,j}$, the amount of practice of task $i$ during
time slot $j$ (e.g., a day in a sequence of days), are organized into a tableau like the following:

|          | $1$   | $2$ |$\ldots$|$m$|
|:--------:|:-------:|:-------:|:--------:|:--------:|
|$item_1$  | $P_{1,1}$    | $P_{1,2}$   |   |   $P_{1,m}$      |
|$item_2$  | $P_{2,1}$    | $P_{2,2}$   |  |    $P_{2,1}$     |
|$\vdots$  |          |   |    $\ddots$     |       |
|$item_{n}$| $P_{n,1}$|$P_{n,2}$   |        | $P_{n,m}$       |

where $n =NITEMS$, and $m = NSLOTS$.

Rows are the items to be studied and columns are time 
slots (e.g., days). Constraints are expressed in terms of sums over
rows and columns, or in the 
case of the density constraint, over a series of adjacent columns.

### Indices
\begin{align*}
i \in \{0, 1,  \ldots, NITEMS-1 \} && \text{Index over items}\\
t \in \{0, 1, \ldots, NSLOTS-1 \} && \text{Index over time slots} \\
t_\delta \in \{0, 1, \ldots, WINSZ\} && \text{Index over time window}
\end{align*}

See below for the use of $t_\delta$ and $WINSZ$ to express 
limits on practice "density".

### Data

\begin{align*}
NITEMS             && \text{Number of items to schedule} \\
NSLOTS             && \text{Number of time slots available} \\
ERGUSED_i          && \text{Amount of mental "energy" required per chunk of practice time for task i} \\
ERGEVAIL_t        && \text{Amount of mental "energy" available per day}\\
TIMEAVAIL_t       && \text{Amount of time resource available during a training slot $t$ (chunks per day)} \\
TIMEPER_i         && \text{Practice time needed to master item $i$} \\
MINPERWIN_i  && \text{Minimum number of training slots for item $i$ within a window (days)} \\
MAXPERWIN_i  && \text{Maximum number of training slots for item $i$ within a window (days)} \\
WINSZ            && \text{Used to express minimum and maximum "density" of practice days (days)}\\
\end{align*}


### Variables
\begin{align*}
P_{i,t}              && \text{Amount of practice in time slot $t$ for item $i$ (chunks of tudy)}\\
ERG_{t}           && \text{Energy expended on a given day (really a macro not a variable)}\\
ERGDEC_t     && \text{Amount by which energy expenditure decreased at $t$ from $t-1$}\\
ERGINC_t     && \text{Amount by which energy expenditure increased at $t$ from $t-1$}\\
TIMESOFAR_{i,t}      && \text{Amount of practice for item $i$ up to and including time $t$}\\
\end{align*}

### Objective

Minimize $\sum_{i, t} TIMEPER_i P_{i,t} + \sum_{t=1}{(ERGINC_t - ERGDEC_t)} + \sum_{i,t} TIMESOFAR_{i,t}$

### Constraints
\begin{align*}
 \sum_{i}{P_{i,t}} &\leq TIMEAVAIL_t\ \forall\ t & \text{C1: No more time allocated to time slot than available} \\
 \sum_{t}{P_{i,t}} &\geq TIMEPER_i\   \forall\ i & \text{C2: Practice time greater than required to learn item} \\
 \sum_{i}{P_{i,t + t_\delta}}  &\geq MINPERWIN\ \forall i, t \forall t_\delta 
  & \text{C3: Practice spacing - minimum amount of practice in window} \\
 \sum_{i}{P_{i,t + t_\delta}}  &\leq MAXPERWIN\ \forall i, t
  & \text{C4: Practice spacing for rest - maximum amount of practice in window}\\
 TIMESOFAR_{i,t} &= \sum_i{\sum_{t0=0 \ldots t}{P_{i,t0}}}\ \forall i,t 
   & \text{C5: Definition. Used in objective to bias toward finishing earlier}\\
 ERG_{t} &= \sum_i{ERGUSED_i \times P_{i,t}}\ \forall t
   & \text{C6: Definition. Used in definition of increase/decrease for smoothing}\\
 ERGINC_{t+1} - ERGDEC_{t+1} &= ERG_{t+1} - ERG_{t}  
   & \text{C7: Smooth energy. Minimization means only one non-zero}
\end{align*}

### Discussion

Note I had to introduce the $WINSZ$ parameter in order to have a way
to express (C3) and (C4). Right now this is something that needs
to be specified as part of the configuration of the model. It's a bit
unnatural.

When I first started experimenting I noticed that these constraints could
result in practice schedules "pushed to the right" with low density
slots to the left of the schedule because I had not expressed the 
idea of optimizing the amount of calendar time spent. As a heuristic,
I decided to incorporate another variable: $TIMESOFAR_{i,t}$ defined as the sum
of practice time for each item $i$ up to time $t$. The idea
was to maximize the sum of these times as a "timeliness" metric. It ends
up penalizing late completion for each task.

The time window is sort of awkward and creates some odd scheduling artifacts.
Next I'm going to try different sorts of energy surpluses/deficits. where there
are constraints that prevent it from going too high or too low. For each
practice item a day of rest will deplete the "learning" banked, while a day of
practice will increase it, but only within a threshold. Equivalent to production
smoothing in other problems I've seen.
